(c) 2016 - present. Enplus Advisors, Inc.

In [1]:
import numpy as np
import pandas as pd

# Merge, Join, and Pivot

## Revisit our familiar DataFrame

In [2]:
symbols = ['AAPL', 'MSFT', 'IBM', 'YHOO']
df = pd.DataFrame({
    'ticker': symbols,
    'date': ['2015-12-31'] * len(symbols),
    'open': [426.23, 42.3, 101.65, 35.53]
})
df

,ticker,date,open
0,AAPL,2015-12-31,426.23
1,MSFT,2015-12-31,42.30
2,IBM,2015-12-31,101.65
3,YHOO,2015-12-31,35.53


## Merge Join

In [3]:
close_prc = df['open'] + np.random.randn(df.shape[0])
df2 = pd.DataFrame({
    'ticker': symbols + ['MANT'],
    'close': np.append(close_prc.values, [77.2])
})
df2

,ticker,close
0,AAPL,426.487633
1,MSFT,43.005895
2,IBM,101.182734
3,YHOO,35.058956
4,MANT,77.200000


In [4]:
pd.merge(df, df2, on='ticker')

,ticker,date,open,close
0,AAPL,2015-12-31,426.23,426.487633
1,MSFT,2015-12-31,42.30,43.005895
2,IBM,2015-12-31,101.65,101.182734
3,YHOO,2015-12-31,35.53,35.058956


In [5]:
pd.merge(df, df2, on='ticker', how='right')

,ticker,date,open,close
0,AAPL,2015-12-31,426.23,426.487633
1,MSFT,2015-12-31,42.30,43.005895
2,IBM,2015-12-31,101.65,101.182734
3,YHOO,2015-12-31,35.53,35.058956
4,MANT,NaN,NaN,77.200000


## Merge Join

In [6]:
df_close = pd.DataFrame(symbols)
df_close.index.name = 'ticker'
df_close.reset_index(inplace=True)

In [7]:
pd.merge(df, df_close, left_index=True, right_on='ticker')

,ticker,ticker_x,date,open,ticker_y,0
0,0,AAPL,2015-12-31,426.23,0,AAPL
1,1,MSFT,2015-12-31,42.30,1,MSFT
2,2,IBM,2015-12-31,101.65,2,IBM
3,3,YHOO,2015-12-31,35.53,3,YHOO


## Pivot

In [8]:
df

,ticker,date,open
0,AAPL,2015-12-31,426.23
1,MSFT,2015-12-31,42.30
2,IBM,2015-12-31,101.65
3,YHOO,2015-12-31,35.53


In [9]:
df['close'] = np.random.randn(4)
df1 = df.reset_index().set_index(['ticker', 'date'])
df1

,,index,open,close
ticker,date,,,
AAPL,2015-12-31,0,426.23,0.383254
MSFT,2015-12-31,1,42.30,-0.455515
IBM,2015-12-31,2,101.65,0.107258
YHOO,2015-12-31,3,35.53,1.794179


In [10]:
df_long = df1.stack().reset_index().rename(columns={'level_2': 'variable', 0: 'value'})
df_long

,ticker,date,variable,value
0,AAPL,2015-12-31,index,0.000000
1,AAPL,2015-12-31,open,426.230000
2,AAPL,2015-12-31,close,0.383254
3,MSFT,2015-12-31,index,1.000000
4,MSFT,2015-12-31,open,42.300000
5,MSFT,2015-12-31,close,-0.455515
6,IBM,2015-12-31,index,2.000000
7,IBM,2015-12-31,open,101.650000
8,IBM,2015-12-31,close,0.107258
9,YHOO,2015-12-31,index,3.000000


In [11]:
res = []
for _, df, in df_long.groupby('ticker'):
    tmp = df[df.value == df.value.max()]
    res.append(tmp)
pd.concat(res, ignore_index=True)

,ticker,date,variable,value
0,AAPL,2015-12-31,open,426.23
1,IBM,2015-12-31,open,101.65
2,MSFT,2015-12-31,open,42.30
3,YHOO,2015-12-31,open,35.53


In [12]:
def keep_max(df):
    return df[df.value == df.value.max()]
df_long.groupby('ticker').apply(keep_max)

,,ticker,date,variable,value
ticker,,,,,
AAPL,1,AAPL,2015-12-31,open,426.23
IBM,7,IBM,2015-12-31,open,101.65
MSFT,4,MSFT,2015-12-31,open,42.30
YHOO,10,YHOO,2015-12-31,open,35.53


In [13]:
df_long.drop_duplicates('ticker', keep='first')

,ticker,date,variable,value
0,AAPL,2015-12-31,index,0.0
3,MSFT,2015-12-31,index,1.0
6,IBM,2015-12-31,index,2.0
9,YHOO,2015-12-31,index,3.0
